In [23]:
import pandas as pd
# import numpy as np

dtype = {
#     'srcip': 'str',
    'sport': 'int32',
#     'dstip': 'str',
    'dsport': 'int32',
    'proto': 'str',
    'state': 'str',
    'dur': 'float64',
    'sbytes': 'int32',
    'dbytes': 'int32',
    'sttl': 'int32',
    'dttl': 'int32',
    'sloss': 'int32',
    'dloss': 'int32',
    'service': 'str',
    'Sload': 'float64',
    'Dload': 'float64',
    'Spkts': 'int32',
    'Dpkts': 'int32',
    'swin': 'int32',
    'dwin': 'int32',
    'stcpb': 'int32',
    'dtcpb': 'int32',
    'smeansz': 'int32',
    'dmeansz': 'int32',
    'trans_depth': 'int32',
    'res_bdy_len': 'int32',
    'Sjit': 'float64',
    'Djit': 'float64',
#     'Stime': '',
#     'Ltime': '',
    'Sintpkt': 'float64',
    'Dintpkt': 'float64',
    'tcprtt': 'float64',
    'synack': 'float64',
    'ackdat': 'float64',
    'is_sm_ips_ports': 'int8',
    'ct_state_ttl': 'int32',
    'ct_flw_http_mthd': 'Int64',
    'is_ftp_login': 'Int64',
    'ct_ftp_cmd': 'Int64',
    'ct_srv_src': 'int32',
    'ct_srv_dst': 'int32',
    'ct_dst_ltm': 'int32',
    'ct_src_ltm': 'int32',
    'ct_src_dport_ltm': 'int32',
    'ct_dst_sport_ltm': 'int32',
    'ct_dst_src_ltm': 'int32',
    'attack_cat': 'str',
    'Label': 'int8',
}
date_cols = ['Stime', 'Ltime']
fields = ['ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'Stime', 'Ltime']
fields2 = ['is_ftp_login', 'ct_ftp_cmd', 'Stime', 'Ltime']
fields3 = ['ct_ftp_cmd', 'Stime', 'Ltime']
fields4 = [
    'srcip',
    'sport',
    'dstip',
    'dsport',
    'state',
    'dur',
    'sbytes',
    'dbytes',
    'sttl',
    'dttl',
    'sloss',
    'dloss',
    'service',
    'Sload',
    'Dload',
    'Spkts',
    'Dpkts',
    'swin',
    'dwin',
    'stcpb',
    'dtcpb',
    'smeansz',
    'dmeansz',
    'trans_depth',
    'res_bdy_len',
    'Sjit',
    'Djit',
    'Stime',
    'Ltime',
    'Sintpkt',
    'Dintpkt',
    'tcprtt',
    'synack',
    'ackdat',
    'is_sm_ips_ports',
    'ct_state_ttl',
    'ct_flw_http_mthd',
    'is_ftp_login',
    'ct_ftp_cmd',
    'ct_srv_src',
    'ct_srv_dst',
#     'ct_dst_ltm',
#     'ct_src_ltm',
    'ct_src_dport_ltm',
    'ct_dst_sport_ltm',
    'ct_dst_src_ltm',
    'attack_cat',
    'Label',
]
fields5 = ['attack_cat', 'Stime', 'Ltime']


def strip_int(text):
    return int(text.strip(' '))

def ip_to_int(ip):
    z = ''
    parts = ip.split('.')
    for prt in range(3):
        if len(parts[prt]) == 3:
            z = parts[prt] + z
        elif len(parts[prt]) == 2:
            z = '0' + parts[prt] + z
        else:
            z = '00' + parts[prt] + z
    return int(z)

def encode_attack_cat(cat):
    return 1

converters = {
    'srcip': ip_to_int,
    'dstip': ip_to_int,
#     'attack_cat': encode_attack_cat
#     'ct_ftp_cmd': strip_int
}

df = pd.read_csv(
    '../data/UNSW-NB15-BALANCED-TRAIN.csv',
#     dtype=dtype,
    parse_dates=date_cols,
    usecols=fields5,
    skipinitialspace=True,
    converters=converters
)
df.head()

,Stime,Ltime,attack_cat
0,1421950270,1421950270,NaN
1,1421944870,1421944874,NaN
2,1424251725,1424251725,Generic
3,1424233686,1424233686,Generic
4,1424244324,1424244324,NaN


In [24]:
df.attack_cat.unique()

array([nan, 'Generic', 'Fuzzers', 'Exploits', 'DoS', 'Fuzzers ',
       'Reconnaissance ', 'Backdoor', 'Reconnaissance', 'Analysis',
       'Shellcode', 'Shellcode ', 'Backdoors', 'Worms'], dtype=object)

In [13]:
df.describe()

,srcip,dstip
count,4.497960e+05,4.497960e+05
mean,9.646951e+07,1.284399e+08
std,8.570929e+07,1.231686e+07
min,1.660590e+05,2.240000e+02
25%,1.660590e+05,1.261711e+08
50%,1.760452e+08,1.261711e+08
75%,1.760452e+08,1.261711e+08
max,2.411682e+08,2.411682e+08


In [38]:
df.values

array([['59.166.0.1', '55490', '149.171.126.3', ..., 1, nan, 0],
       ['59.166.0.9', '26756', '149.171.126.8', ..., 8, nan, 0],
       ['175.45.176.1', '1043', '149.171.126.14', ..., 17, 'Generic', 1],
       ...,
       ['59.166.0.9', '31192', '149.171.126.8', ..., 1, nan, 0],
       ['175.45.176.1', '1043', '149.171.126.18', ..., 41, 'Generic', 1],
       ['59.166.0.8', '56828', '149.171.126.2', ..., 1, nan, 0]],
      dtype=object)

In [39]:
X = df.drop(columns=['attack_cat', 'Label'])
y = df['Label']

df.dtypes

srcip                object
sport                object
dstip                object
dsport               object
proto                object
state                object
dur                 float64
sbytes                int32
dbytes                int32
sttl                  int32
dttl                  int32
sloss                 int32
dloss                 int32
service              object
Sload               float64
Dload               float64
Spkts                 int32
Dpkts                 int32
swin                  int32
dwin                  int32
stcpb                 int32
dtcpb                 int32
smeansz               int32
dmeansz               int32
trans_depth           int32
res_bdy_len           int32
Sjit                float64
Djit                float64
Stime                object
Ltime                object
Sintpkt             float64
Dintpkt             float64
tcprtt              float64
synack              float64
ackdat              float64
is_sm_ips_ports     

In [46]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
processed_df = ohe.fit_transform(df)
processed_df[0]

<1x1987485 sparse matrix of type '<class 'numpy.float64'>'
	with 49 stored elements in Compressed Sparse Row format>

In [33]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = DecisionTreeClassifier()
le = preprocessing.LabelEncoder()
X_train = 
model.fit(X_train, y_train)
predictions = model.predict(X_test)

score = accuracy_score(y_test, predictions)
score

ValueError: could not convert string to float: '175.45.176.1'